In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/elomia

Mounted at /content/drive
/content/drive/MyDrive/elomia


In [2]:
!pip install -U sentence-transformers
!pip install transformers
!pip install datasets

     |████████████████████████████████| 79 kB 3.5 MB/s 
     |████████████████████████████████| 3.8 MB 25.7 MB/s 
     |████████████████████████████████| 1.2 MB 81.1 MB/s 
     |████████████████████████████████| 77 kB 8.8 MB/s 
     |████████████████████████████████| 596 kB 73.5 MB/s 
     |████████████████████████████████| 6.5 MB 56.4 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=949c9cf9daf833bf58093d3f84580d512d489f6d8fdc2c5b73cd3ccebc86a2c8
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 83.4 MB/s 
     

In [3]:
import os
import glob

# Convert data to json format

In [4]:
info_keys = ["Description", "URL","Intro message", "Follow-up messages"]

desc_dict = {}
for f_path in glob.glob(os.path.join("exercises_descriptions","*")):
    ex_type = f_path.split("/")[-1]
    desc_dict[ex_type] = {}
    with open(f_path) as file:
        s = file.read().replace('\n', '')

        for index, info_k in enumerate(info_keys):
          end_k = info_keys[index + 1] if index < len(info_keys) - 1 else ""
          cropped_s = s[s.find(info_k)+len(info_k):s.rfind(end_k)]
          cropped_s = cropped_s.replace(": ","") # remove extra chars

          desc_dict[ex_type][info_k] = cropped_s

test_ex_type = "ideal_relationship"
print(desc_dict[test_ex_type]["Description"])
print(desc_dict[test_ex_type]["URL"])
print(desc_dict[test_ex_type]["Intro message"])
print(desc_dict[test_ex_type]["Follow-up messages"])

Медитация-визуализация. Подходит людям, с проблемами в отношениях. Проблемы могут быть очень разнымиклиент не может найти партнера, клиент сомневается, что нынешний партнер ему подходит, клиент хочет улучшить качество своих отношений.   
https://exercise.elomia.com/#/:token/RelationshipsMeditation       
This short meditation will help you understand what ideal relationships look like for you and find ways to improve your current situation       
$user, did you feel optimistic after this meditation? Why? | $user, so, can you describe your ideal relationships now?


# Calculate embeddings

In [5]:
from transformers import AutoTokenizer, AutoModelWithLMHead

from sentence_transformers import SentenceTransformer, util, InputExample, losses
from sentence_transformers.cross_encoder import CrossEncoder


# Pre-trained cross encoder
cross_encoder = CrossEncoder('cross-encoder/stsb-distilroberta-base')

model_name = "sentence-transformers/all-mpnet-base-v2" #"distiluse-base-multilingual-cased-v1" - has problems with finutuning 

sentence_model = SentenceTransformer(model_name) # multilingual ?! 0_0

Downloading:   0%|          | 0.00/607 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/148 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def compute_desc_dict_embs(desc_dict):
    for key, value in desc_dict.items():
        desc_txt = value["Description"]
        intr_msg_txt = value["Intro message"]

        desc_embs = sentence_model.encode(desc_txt)
        intr_msg_embs = sentence_model.encode(intr_msg_txt)

        desc_dict[key]["desc_embs"] = desc_embs
        desc_dict[key]["intr_msg_embs"] = intr_msg_embs

        distance = util.cos_sim(desc_embs, intr_msg_embs).numpy()[0][0]
        print(key)
        print(desc_txt)
        print(intr_msg_txt)
        print(distance)

compute_desc_dict_embs(desc_dict)

ideal_relationship
Медитация-визуализация. Подходит людям, с проблемами в отношениях. Проблемы могут быть очень разнымиклиент не может найти партнера, клиент сомневается, что нынешний партнер ему подходит, клиент хочет улучшить качество своих отношений.   
This short meditation will help you understand what ideal relationships look like for you and find ways to improve your current situation       
0.1547121
after_break
Утренняя медитация. Медитация подходит тем, кто страдает из-за разбитого сердца и не может найти в себе силы двигаться дальше. Медитация должна зарядить надеждой и энергией на новый день. Важно давать ее с утра или советовать клиенту зайти завтра утром за этой медитацией.   
I recommend you to try this meditation. It helps to recover after the painful break-up and start a new life without negative thoughts.  | Try this short meditation. It helps to recover after a painful break-up and to get rid of any negative thoughts       
0.05640359
love_yourself
Медитация, которая

In [7]:
from utils import get_cleared_exercise_df

exercise_df, max_phrase_len = get_cleared_exercise_df("data")
exercise_df.head()

,Монолог,Упражнение,Уверенность,Ок/Не ок,Unnamed: 3
0,i`ve been feeling stressed out . i have been f...,CalmDownExercise,-0.012248,True,NaN
1,"two birds on a wire . one tries to fly away , ...",CalmDownExercise,-0.378436,False,NaN
2,"help means more needs than other students , an...",CalmDownExercise,-0.031085,False,NaN
3,"i`ve been feeling sad , lonely , and anxious ....",CalmDownExercise,-0.624913,False,NaN
4,i`m usually response to her complaints because...,CalmDownExercise,-0.426002,False,NaN


In [8]:
data_embs = sentence_model.encode(exercise_df["Монолог"])

In [9]:
data_embs.shape

(655, 768)

In [10]:
desc_dict.keys()

dict_keys(['ideal_relationship', 'after_break', 'love_yourself', 'lose_weight', 'calm_down', 'fall_asleep', 'increase_happines', 'make_a_decision', 'stop_anxiety', 'boost_productivity', 'stop_procrastinating'])

# Sentiment analysis


In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
# prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
# print(prediction)

# sentiment_tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
# sentiment_model = AutoModelWithLMHead.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")


Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

In [ ]:
# print(exercise_df["Монолог"].tolist())
prediction = classifier(exercise_df["Монолог"].tolist())
print(prediction)

In [ ]:
# prediction = classifier("i feel like i’m so full of people at work since i’m a teacher , but when i get home by myself , i don’t know what to do with myself . i don’t feel like i have enough hobbies to spend my time on . how should i deal with anxiety ?")

# print(prediction)

[[{'label': 'sadness', 'score': 0.0003687519347295165}, {'label': 'joy', 'score': 0.9980521202087402}, {'label': 'love', 'score': 0.0009873679373413324}, {'label': 'anger', 'score': 0.00021856663806829602}, {'label': 'fear', 'score': 0.00013215257786214352}, {'label': 'surprise', 'score': 0.00024109375954139978}]]


In [ ]:
import numpy as np
emotions = ["sadness", "joy", "love", "anger", "fear", "surprise"]  # be careful here ! order can be changeable 

def inner_apply(x):
    return list(map(lambda inner_x: inner_x["score"], x))

def inner_just_score_apply(x): # for variance 
    return list(map(lambda inner_x: inner_x["score"], x))

exercise_df["emotions"] = list(map(lambda x: emotions[np.argmax(inner_apply(x))], prediction))
exercise_df["emotions_val"] = list(map(lambda x: inner_apply(x)[np.argmax(inner_apply(x))], prediction)) # a little bit crazy
exercise_df["emotions_variance"] = list(map(lambda x: np.var(inner_just_score_apply(x)), prediction))
exercise_df["emotions_preds"] = prediction

In [14]:
exercise_df["Emotions_var"]

KeyError: ignored

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 300
test_df = exercise_df[["Монолог", 'Ок/Не ок', "emotions", "emotions_val", "emotions_variance", "emotions_preds"]]
# test_df[test_df['Ок/Не ок']==True]["Emotions"].value_counts()

test_df[(test_df['Ок/Не ок']==True) & (test_df['emotions']=="joy")].head(100)

In [ ]:
test_df[(test_df['Ок/Не ок']==False) & (test_df['emotions']=="joy")].head(100)

In [ ]:
import numpy as np 

def calc_acc(exercise_df, data_embs, desc_dict):
    difference_array = util.cos_sim(data_embs, desc_dict["calm_down"]["intr_msg_embs"]) #intr_msg_embs #"desc_embs"
    difference_array = difference_array.flatten()
    # find the index of minimum element from the array

    index = difference_array.argmin()
    idx = np.argpartition(difference_array, 5)
    # print(idx)

    test_df = exercise_df.iloc[idx[0:100]]

    test_len = len(test_df[(test_df['Упражнение'] == "CalmDownExercise")])
    print(test_len/len(test_df))

    test_len = len(test_df[(test_df['Упражнение'] == "CalmDownExercise") & (test_df["Ок/Не ок"] == True)])
    
    test_len2 = len(exercise_df[(exercise_df['Упражнение'] == "CalmDownExercise") & (exercise_df["Ок/Не ок"] == True)])

    print(test_len/len(test_df))
    print(test_len/test_len2)

calc_acc(exercise_df, data_embs, desc_dict)

In [ ]:
import itertools
from itertools import product

# https://www.geeksforgeeks.org/python-program-to-get-all-unique-combinations-of-two-lists/

def get_all_pairs(list_1, list_2=None):
    if list_2 is None:
      return list(itertools.combinations(list_1, 2))
    else:
      return list(itertools.product(list_1, list_2))

In [ ]:
from torch.utils.data import DataLoader, Dataset

c_exercise_df = exercise_df.copy()

c_exercise_df["alt_label"] = np.zeros((len(c_exercise_df))).tolist()  #OUTPUT MODEL SHAPE
c_exercise_df["is_ok_calm"] = np.zeros((len(c_exercise_df))).tolist()  #OUTPUT MODEL SHAPE

desc_keys = list(desc_dict.keys())
desc_values = list(map(lambda x:x["Intro message"], desc_dict.values()))

def foo(x):
  pairs = get_all_pairs([x["Монолог"]], desc_values)
  preds = cross_encoder.predict(pairs)
  x["alt_label"] = desc_keys[np.argmax(preds)]

  if x['Упражнение'] == "CalmDownExercise" and x["Ок/Не ок"] == True:
      x["is_ok_calm"] = 1
  else:
      x["is_ok_calm"] = 0

  return x

c_exercise_df = c_exercise_df.apply(lambda x: foo(x), axis=1)

In [ ]:
len(c_exercise_df)

In [ ]:
len(c_exercise_df[(c_exercise_df["Упражнение"] == "CalmDownExercise") & (c_exercise_df["alt_label"] == "calm_down")]) / len(c_exercise_df)

In [ ]:
ok_calm_len = len(c_exercise_df[(c_exercise_df["Упражнение"] == "CalmDownExercise") &
                  (c_exercise_df["Ок/Не ок"] == True)]) 

len(c_exercise_df[(c_exercise_df["Упражнение"] == "CalmDownExercise") &
                  (c_exercise_df["alt_label"] == "calm_down") & 
                  (c_exercise_df["Ок/Не ок"] == True)]) / ok_calm_len

In [ ]:
len(c_exercise_df[(c_exercise_df["Упражнение"] == "CalmDownExercise") &
                  (c_exercise_df["Ок/Не ок"] == True)]) / len(c_exercise_df)

In [ ]:
c_exercise_df.to_csv("result_with_alt_labels.tsv", sep='\t')

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 100

inc_hap_test_df = c_exercise_df[(c_exercise_df["Упражнение"] == "IncreaseHappinessExercise")]
stop_anx_test_df = c_exercise_df[(c_exercise_df["Упражнение"] == "StopAnxietyExercise")]

inc_hap_test_df.head(50)
# c_exercise_df["Упражнение"].unique()

In [ ]:
c_exercise_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(c_exercise_df.sample(frac=1), test_size=0.2)
print(len(train_df[train_df["is_ok_calm"]==1])/len(train_df))
print(len(test_df[test_df["is_ok_calm"]==1])/len(test_df))

In [ ]:
positive_train_texts = list(train_df[train_df["is_ok_calm"] == 1]["Монолог"])
negative_train_texts = list(train_df[train_df["is_ok_calm"] == 0]["Монолог"])

# print(len(positive_train_texts))
# print()

positive_test_texts = list(test_df[test_df["is_ok_calm"] == 1]["Монолог"])
negative_test_texts = list(test_df[test_df["is_ok_calm"] == 0]["Монолог"])

# pos_train_pairs = get_all_pairs(positive_train_texts)
# neg_train_pairs = get_all_pairs(positive_train_texts, negative_train_texts)

# pos_test_pairs = get_all_pairs(positive_test_texts)
# neg_test_pairs = get_all_pairs(positive_test_texts, negative_test_texts)


# ADD DESC EMBS ! ! ! ! ! Repeated data
# for _ in range(4): 
pos_train_pairs = get_all_pairs(positive_train_texts, [desc_dict["calm_down"]["Intro message"]])

pos_test_pairs = get_all_pairs(positive_test_texts, [desc_dict["calm_down"]["Intro message"]])

neg_train_pairs = get_all_pairs(negative_train_texts, [desc_dict["lose_weight"]["Intro message"]]) # choose random category

neg_test_pairs = get_all_pairs(negative_test_texts, [desc_dict["lose_weight"]["Intro message"]])

In [ ]:
import random

random.shuffle(pos_train_pairs)
random.shuffle(pos_test_pairs)
random.shuffle(neg_train_pairs)
random.shuffle(neg_test_pairs)

neg_train_pairs = neg_train_pairs[0:len(pos_train_pairs)]
neg_test_pairs = neg_test_pairs[0:len(pos_test_pairs)]

# print(len(neg_train_pairs))
# print(len(neg_test_pairs))

In [ ]:
pos_train_in_exs = list(map(lambda x: InputExample(texts = x, label=0.9), pos_train_pairs))
neg_train_in_exs = list(map(lambda x: InputExample(texts = x, label=-0.9), neg_train_pairs))

pos_test_input_examples = list(map(lambda x: InputExample(texts = x, label=0.9), pos_test_pairs))
neg_test_input_examples = list(map(lambda x: InputExample(texts = x, label=-0.9), neg_test_pairs))

In [ ]:
from sentence_transformers import evaluation

test_list = pos_test_input_examples + neg_test_input_examples
random.shuffle(test_list)
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_list)

In [ ]:
positive_test_texts

# Define your train dataset, the dataloader and the train loss
train_examples = pos_train_in_exs + neg_train_in_exs
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
train_loss = losses.CosineSimilarityLoss(sentence_model)

# # #Tune the model

sentence_model.fit(train_objectives=[(train_dataloader, train_loss)],
                   evaluator=evaluator, 
                   epochs=10, 
                   warmup_steps=10, 
                   evaluation_steps=100)

In [ ]:
compute_desc_dict_embs(desc_dict)
data_embs = sentence_model.encode(exercise_df["Монолог"])
calc_acc(exercise_df, data_embs, desc_dict)

ideal_relationship
Медитация-визуализация. Подходит людям, с проблемами в отношениях. Проблемы могут быть очень разнымиклиент не может найти партнера, клиент сомневается, что нынешний партнер ему подходит, клиент хочет улучшить качество своих отношений.   
This short meditation will help you understand what ideal relationships look like for you and find ways to improve your current situation       
0.8371941
after_break
Утренняя медитация. Медитация подходит тем, кто страдает из-за разбитого сердца и не может найти в себе силы двигаться дальше. Медитация должна зарядить надеждой и энергией на новый день. Важно давать ее с утра или советовать клиенту зайти завтра утром за этой медитацией.   
I recommend you to try this meditation. It helps to recover after the painful break-up and start a new life without negative thoughts.  | Try this short meditation. It helps to recover after a painful break-up and to get rid of any negative thoughts       
0.79446054
love_yourself
Медитация, которая

In [ ]:
print(len(bert_model.bert.encoder.layer))
for i in range(len(bert_model.bert.encoder.layer)):
    bert_model.bert.encoder.layer[i].trainable = i >= 10

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np

# Pre-trained cross encoder
model = CrossEncoder('cross-encoder/stsb-distilroberta-base')

In [ ]:
model.predict(pos_train_pairs)

array([0.9093854 , 0.90951174, 0.91026735, 0.9117384 , 0.91024435,
       0.9072385 , 0.90956414, 0.91026735, 0.90670365, 0.90711325,
       0.9081152 , 0.90799314, 0.9095646 , 0.9102143 , 0.91078395,
       0.9083209 , 0.9087599 , 0.90882987, 0.9081163 , 0.9090497 ,
       0.9086132 , 0.9085214 , 0.90837806, 0.90755355, 0.91178113,
       0.9077815 , 0.9093706 , 0.91184604, 0.90620106, 0.90728503,
       0.9060384 , 0.9053412 , 0.9120919 , 0.9116667 , 0.9097983 ,
       0.9072571 , 0.90761536, 0.9070839 , 0.9104627 , 0.911119  ,
       0.9097983 , 0.90713006, 0.91078395, 0.9054992 , 0.9043244 ,
       0.90999126, 0.90698135, 0.90466964, 0.90522015, 0.90687686,
       0.9131332 , 0.91170067, 0.9084302 , 0.90551144, 0.9097506 ,
       0.905111  , 0.90867215, 0.90675545, 0.91056764, 0.910403  ,
       0.9057347 , 0.9081148 , 0.9053412 , 0.90867215, 0.90988684,
       0.90775585, 0.9066463 , 0.9090575 , 0.9051622 , 0.90986097,
       0.9073814 , 0.90775007, 0.9099426 , 0.90950716, 0.90582

In [ ]:
model.predict(neg_train_pairs)

array([0.00047435, 0.00048251, 0.00050098, 0.00048291, 0.00048463,
       0.00048637, 0.00046961, 0.00047662, 0.00048385, 0.00048709,
       0.00052335, 0.00048688, 0.00047529, 0.0004884 , 0.00047536,
       0.00048269, 0.00047111, 0.0005066 , 0.00047792, 0.0004772 ,
       0.00047371, 0.00047035, 0.0004722 , 0.00048881, 0.00047716,
       0.00047157, 0.00047469, 0.0004892 , 0.00047067, 0.00047666,
       0.00047329, 0.00048123, 0.00047762, 0.00048164, 0.00048862,
       0.00049348, 0.00051849, 0.00048841, 0.00047476, 0.00049406,
       0.00047346, 0.00047515, 0.00048402, 0.00048933, 0.00047339,
       0.00047702, 0.00047732, 0.00047218, 0.00050339, 0.00047638,
       0.00048064, 0.00048955, 0.00050807, 0.0004721 , 0.00047129,
       0.00047889, 0.0004926 , 0.00049728, 0.00047621, 0.00048058,
       0.00049535, 0.00047467, 0.00048064, 0.00048493, 0.0005009 ,
       0.00047205, 0.00049986, 0.00047895, 0.00048328, 0.0005008 ,
       0.00047228, 0.00047853, 0.00047136, 0.00046978, 0.00048

In [ ]:
def calc_acc2(exercise_df, desc_dict, model):
    df = exercise_df.copy()#.sample(frac=1)
    pairs = get_all_pairs(df["Монолог"], [desc_dict["calm_down"]["Intro message"]])
    preds = model.predict(pairs)
    df["similarity"] = preds

    df.sort_values(by=["similarity"], axis=0, ascending=True)
                          
    test_df = df[0:300]

    test_len = len(test_df[(test_df['Упражнение'] == "CalmDownExercise")])
    print(test_len/len(test_df))

    test_len = len(test_df[(test_df['Упражнение'] == "CalmDownExercise") & (test_df["Ок/Не ок"] == True)])
    test_len2 = len(df[(df['Упражнение'] == "CalmDownExercise") & (df["Ок/Не ок"] == True)])

    print(test_len/len(test_df))
    print(test_len/test_len2)

calc_acc2(exercise_df, desc_dict, model)

0.86
0.30666666666666664
0.4946236559139785


In [ ]:
pos_train_in_exs = list(map(lambda x: InputExample(texts = x, label=0.9), pos_train_pairs))
neg_train_in_exs = list(map(lambda x: InputExample(texts = x, label=0.0001), neg_train_pairs))

# Define your train dataset, the dataloader and the train loss
train_examples = pos_train_in_exs + neg_train_in_exs
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
# # #Tune the model

model.fit(train_dataloader=train_dataloader,
                   epochs=10, 
                   warmup_steps=10, 
                   evaluation_steps=100)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
calc_acc2(exercise_df, desc_dict, model)

0.86
0.30666666666666664
0.4946236559139785


In [ ]:
similarity_scores = model.predict(sentence_combinations)

In [ ]:
print(similarity_scores)

[0.6732372  0.3410255  0.00542466 0.0756935  0.00525379 0.00536814
 0.06676244 0.00534824 0.00516718]
